# Trading & Positions

[index](./index.ipynb) |
[accounts](./accounts.ipynb) |
[balances](./balances.ipynb)

In [1]:
# SAXO related imports
from saxo_openapi import API
from saxo_openapi.exceptions import OpenAPIError
# endpoints for trading en portfolio service group
import saxo_openapi.endpoints.trading as tr
import saxo_openapi.endpoints.portfolio as pf
# to make life easier
from saxo_openapi.contrib.session import account_info

import json
import juputil
from pprint import pprint

import pandas as pd

In [2]:
# The basic setup
token = juputil.read_token()
client = API(access_token=token)
# and fetch our account information
ai = account_info(client)

## Create some orders

SAXO Bank offers a lot of instruments to trade. Not all ordertypes apply for all instruments.

The *saxo_openapi* library offers some classes to construct orderbodies pretty easy. 

In [3]:
from saxo_openapi.contrib.orders import MarketOrder, MarketOrderFxSpot, tie_account_to_order
import saxo_openapi.contrib.orders.onfill as onfill
import saxo_openapi.definitions.orders as OD

### Market order: Short 10k EURUSD

This can be done by using the generic *MarketOrder* class, but also by using *MarketOrderFxSpot*

In [4]:
mo1 = MarketOrder(Uic=21, AssetType=OD.AssetType.FxSpot, Amount=-10000)
# or
mo2 = MarketOrderFxSpot(Uic=21, Amount=-10000)

print("mo1 == mo2: {}\n".format(mo1.data == mo2.data))
pprint(mo1.data)

mo1 == mo2: True

{'Amount': 10000,
 'AmountType': 'Quantity',
 'AssetType': 'FxSpot',
 'BuySell': 'Sell',
 'OrderDuration': {'DurationType': 'DayOrder'},
 'OrderType': 'Market',
 'Uic': 21}


### If done orders ...

Adding a *StopLoss* and/or a *TakeProfit* order after the order got filled? That can be done
pretty easy too. Create one of the possible *onfill* instances by simply passing the *price* where
to take the profit / loss.
Optionally you can override the defaults for for instance the *orderduration*. Check the *saxo_openapi* -documentation for details.

In [5]:
# current price of EURUSD is 1.1215,
# SL: lets close the Short if it reaches 1.1255
# TP: lets take profit if it reaches 1.0945
mo_sl_tp = MarketOrderFxSpot(Uic=21,
                             Amount=-10000,
                             StopLossOnFill=onfill.StopLossDetails(1.1255),
                             TakeProfitOnFill=onfill.TakeProfitDetails(1.0945))
pprint(mo_sl_tp.data)

{'Amount': 10000,
 'AmountType': 'Quantity',
 'AssetType': 'FxSpot',
 'BuySell': 'Sell',
 'OrderDuration': {'DurationType': 'DayOrder'},
 'OrderType': 'Market',
 'Orders': [{'Amount': 10000,
             'AssetType': 'FxSpot',
             'BuySell': 'Buy',
             'OrderDuration': {'DurationType': 'GoodTillCancel'},
             'OrderPrice': 1.0945,
             'OrderType': 'Limit',
             'Uic': 21},
            {'Amount': 10000,
             'AssetType': 'FxSpot',
             'BuySell': 'Buy',
             'OrderDuration': {'DurationType': 'GoodTillCancel'},
             'OrderPrice': 1.1255,
             'OrderType': 'Stop',
             'Uic': 21}],
 'Uic': 21}


### So far ... 

We have an orderbody that represents the instrument the way we want to trade it.

But for SAXO Bank to be able to process that orderbody it needs the *account* for which
the order applies. 

So, the orderbody needs to be 'enriched' with the *AccountKey* of the account.

The *saxo_openapi.contrib.orders* module offers a function *tie_account_to_order*. This
function processes the orderbody and injects the *AccountKey* there where needed.

    ...
    ai = account_info(client)
    ...
    mo = tie_account_to_order(ai.AccountKey, mo_sl_tp)
    r = tr.orders.Order(data=mo)
    rv = client.request(r)
    
This decoupled approach makes it easy to process the order for multiple accounts:

In [6]:
# use tie_account_to_order to inject the AccountKey
mo = tie_account_to_order(ai.AccountKey, mo_sl_tp)
pprint(mo)

{'AccountKey': 'fOA0tvOyQqW2aHpWi9P5bw==',
 'Amount': 10000,
 'AmountType': 'Quantity',
 'AssetType': 'FxSpot',
 'BuySell': 'Sell',
 'OrderDuration': {'DurationType': 'DayOrder'},
 'OrderType': 'Market',
 'Orders': [{'AccountKey': 'fOA0tvOyQqW2aHpWi9P5bw==',
             'Amount': 10000,
             'AssetType': 'FxSpot',
             'BuySell': 'Buy',
             'OrderDuration': {'DurationType': 'GoodTillCancel'},
             'OrderPrice': 1.0945,
             'OrderType': 'Limit',
             'Uic': 21},
            {'AccountKey': 'fOA0tvOyQqW2aHpWi9P5bw==',
             'Amount': 10000,
             'AssetType': 'FxSpot',
             'BuySell': 'Buy',
             'OrderDuration': {'DurationType': 'GoodTillCancel'},
             'OrderPrice': 1.1255,
             'OrderType': 'Stop',
             'Uic': 21}],
 'Uic': 21}


## Place the order ...

Place the order by passing the *mo*- order specification for the market order.

In [7]:
r = tr.orders.Order(data=mo)
rv = client.request(r)
print(json.dumps(rv, indent=2))

{
  "OrderId": "77244780",
  "Orders": [
    {
      "OrderId": "77244781"
    },
    {
      "OrderId": "77244782"
    }
  ]
}


... The marketorder OrderId and in "Orders" [] the OrderId's of the 'IfDone-orders'

## Lets find these orders ...

In [8]:
r = pf.orders.GetOpenOrdersMe()
rv = client.request(r)
pprint(rv)

{'Data': [{'AccountId': '9300675',
           'AccountKey': 'fOA0tvOyQqW2aHpWi9P5bw==',
           'Amount': 10000.0,
           'AssetType': 'FxSpot',
           'BuySell': 'Buy',
           'CalculationReliability': 'Ok',
           'ClientKey': 'fOA0tvOyQqW2aHpWi9P5bw==',
           'CorrelationKey': 'a97e5b1c-951f-447c-882c-35c3f8dfccf6',
           'CurrentPrice': 1.11868,
           'CurrentPriceDelayMinutes': 0,
           'CurrentPriceType': 'Bid',
           'DistanceToMarket': 0.00682,
           'Duration': {'DurationType': 'GoodTillCancel'},
           'IsMarketOpen': True,
           'MarketPrice': 1.11868,
           'OpenOrderType': 'Stop',
           'OrderAmountType': 'Quantity',
           'OrderId': '77244782',
           'OrderRelation': 'Oco',
           'OrderTime': '2019-05-03T15:00:39.490000Z',
           'Price': 1.1255,
           'RelatedOpenOrders': [{'Amount': 10000.0,
                                  'Duration': {'DurationType': 'GoodTillCancel'},
       

Both the *Stop* order and the *Limit* order can be found here along with the *RelatedPositionId*.

## Lets find the position created by the MarketOrder

In [38]:
SRC_OrderId = 77244780
r = pf.positions.PositionsMe()
rv = client.request(r)
#pprint(rv)
clpos = None
for P in rv['Data']:
    if not pos['PositionBase']['CanBeClosed']:
        continue
    if 'SourceOrderId' in P['PositionBase'] and P['PositionBase']['SourceOrderId'] == str(SRC_OrderId):
        pprint(P)
        clpos = P  # and save it in clpos

{'NetPositionId': 'EURUSD__FxSpot',
 'PositionBase': {'AccountId': '9300675',
                  'Amount': -10000.0,
                  'AssetType': 'FxSpot',
                  'CanBeClosed': False,
                  'ClientId': '9300675',
                  'CloseConversionRateSettled': False,
                  'CorrelationKey': 'a97e5b1c-951f-447c-882c-35c3f8dfccf6',
                  'ExecutionTimeClose': '2019-05-03T15:39:38.204750Z',
                  'ExecutionTimeOpen': '2019-05-03T15:00:39.483416Z',
                  'IsMarketOpen': True,
                  'OpenPrice': 1.11855,
                  'RelatedOpenOrders': [],
                  'RelatedPositionId': '219259468',
                  'SourceOrderId': '77244780',
                  'SpotDate': '2019-05-07',
                  'Status': 'Closed',
                  'Uic': 21,
                  'ValueDate': '2019-05-07T00:00:00.000000Z'},
 'PositionId': '219256066',
 'PositionView': {'CalculationReliability': 'Ok',
                

## Close the position

Positions can be closed explicitely and implicitely. Explicite closing can only be done when the account
is set to 'End-Of-Day-Netting'.

NOTE: Only an *explicite close* cancels related OCO orders!


In [22]:
from saxo_openapi.contrib.orders import direction_from_amount
from saxo_openapi.contrib.orders.helper import direction_invert

def close_position(position):
    if not position['PositionBase']['CanBeClosed']:
        raise ValueError("Position already closed")
    
    direction = direction_from_amount(position['PositionBase']['Amount'])
    direction = direction_invert(direction)
    
    data = {'PositionId': position['PositionId'],
            'Orders': [{
                'Uic': position['PositionBase']['Uic'],
                'AssetType': position['PositionBase']['AssetType'],
                'Amount': abs(position['PositionBase']['Amount']),
                'OrderType': 'Market',
                'BuySell': direction,
                'OrderDuration': {
                    'DurationType': 'DayOrder'
                }
            }]}
    return data

In [23]:
pcls = close_position(clpos)
pprint(pcls)
data = tie_account_to_order(ai.AccountKey, pcls)
pprint(data)

{'Orders': [{'Amount': 10000.0,
             'AssetType': 'FxSpot',
             'BuySell': 'Buy',
             'OrderDuration': {'DurationType': 'DayOrder'},
             'OrderType': 'Market',
             'Uic': 21}],
 'PositionId': '219256066'}
{'Orders': [{'AccountKey': 'fOA0tvOyQqW2aHpWi9P5bw==',
             'Amount': 10000.0,
             'AssetType': 'FxSpot',
             'BuySell': 'Buy',
             'OrderDuration': {'DurationType': 'DayOrder'},
             'OrderType': 'Market',
             'Uic': 21}],
 'PositionId': '219256066'}


In [24]:
rc = tr.orders.Order(data=data)
try:
    client.request(rc)
except OpenAPIError as oae:
    print(json.dumps(json.loads(oae.content), indent=2))
else:
    print(json.dumps(rc.response, indent=2))

{
  "Orders": [
    {
      "OrderId": "77245705"
    }
  ]
}
